In [1]:
#%pip install ipyleaflet requests

In [2]:
import requests
from IPython.display import display
from ipyleaflet import Map, DrawControl
from transformers import pipeline

/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [3]:
map_center = [20.0, 0.0]
map = Map(center=map_center, zoom=2, scroll_wheel_zoom=True)

def is_valid_data(data):
    hourly_data = data['hourly']

    for hourly_entry in hourly_data.items():
        values = hourly_entry[1]
        if all(value is None for value in values):
            return False

    return True

def handle_marker(event, action, geo_json):
    coordinates = geo_json['geometry']['coordinates']
    longitude, latitude = str(coordinates[0]), str(coordinates[1])

    # Fetch sea conditions from Open-Meteo API
    url = "https://marine-api.open-meteo.com/v1/marine?latitude=" + latitude + "&longitude=" + longitude + "&hourly=wave_height,wave_direction,wave_period,ocean_current_velocity,ocean_current_direction&daily=wave_height_max,wave_direction_dominant,wave_period_max&timezone=auto"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
            print(data)
        else:
            print("You can't cruise on land!")
    else:
        print("Error fetching data from Open-Meteo API: " + str(response.status_code))

draw_control = DrawControl(
    marker={'shapeOptions': {}},
    polyline={},
    polygon={},
    circle={},
    circlemarker={},
)
map.add_control(draw_control)
draw_control.on_draw(handle_marker)

# Display map
display(map)

Map(center=[20.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

You can't cruise on land!
